<a href="https://colab.research.google.com/github/Preranakh/Personal-Informations-Visualization/blob/main/OrganizeFirebaseDataScript.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pandas openpyxl

In [ ]:
pip install xlsxwriter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 3.1 MB/s eta 0:00:00


In [ ]:
import json
import pandas as pd

def convert_time_to_seconds(time_str):
    mins, secs = 0, 0
    if "Mins" in time_str:
        mins = int(time_str.split("Mins")[0].strip())
        secs = int(time_str.split("Mins")[1].split("Secs")[0].strip().split(":")[1])
    else:
        secs = int(time_str.split("Secs")[0].strip())
    return mins * 60 + secs

with open('database1.json', 'r') as file:
    data = json.load(file)

user_data = data['user_session_data_test']

control_group_ids = [
    "izZlxECbEvbZbaQsiTC2IppTR1D2", "w1bQfQVpS0e2bdMS0mgXpSHqZew2", "Uah5QqbULfOwCEwuKqEmRv7ELx42",
    "l1JBi3rjs7eEzKECNW02rhKkzI12", "kt6wMrsVN1YNwymFsZCPha4eG6w1", "gSznvtvnoeMV3bvm9aMP0uwlUhl2",
    "ZbmqqGFDwmZXkjR9lxJRB8Y6tmd2", "U2HjmAv7KCcfQVHsr7ofVPdaTm82", "n3mtRWzHDRSm9NYqoGxyyM7zyuQ2",
    "Vv3J8PwWWeSWErREMVtrxwkP8rF3", "ogFBk6o71cWw7TQejwgzYWYdetK2", "YOlWWzDFhyUJuCCazPl7vMdPLj23",
    "wrGFkJdPKwNNcOIDSPmN62VGIzC2", "g6RotBsUG0fQVfkSampcMTaWdTE3", "2blzSKV1tkcSFq2f8opLrcM5iP33",
    "EfVmI4iVenXsMAX392YBkIgysH23", "lJNN7pxBDhTulyLFLr70FghZLfN2", "6X2UCYgJSBV0dFWD0yWYodEmNKu2"
]

intervention_group_ids = [
    "3YPGBN3vD3TIqg3Jlrxgu16gj4K3", "ehQq99p6woP538JE31l8s0lcBr62", "NMUVcKxsfbT76avSebXt6fmsKw02",
    "iVMz6gl0vkWNZuTJ64bUXEj1AJm1", "K9IJNZxN3OUNiOn6kFuia77Io9m2", "ttrSxxnr5ROCWCynB2kC2cJgkEr1",
    "ZXkLCiyTgyXJGmL5dXF7Pip5Tpk2", "qf7BdXXKWTNisXiY3dNGynktezC3", "6AfBZNSFoDYtFZoK3v6IoDzYGX23",
    "DMRsCjYfw3Ui2WY7baPfiliLmYm1", "yOwa5JFnOFcPhY68B9JfQ8FXhqN2", "RXrptwB3hBeMx77a0g09or8T7zk2",
    "4N71ZXMaaTcJbEmeUAg1KrZlQNA3", "iXcMJZtl9wXfvb5elAKio2Nrn0D2", "OIf29ayJNJUCcBRExfWG27W9iCh1",
    "okJYh5CBOrbVuzpP7dLOiBWxGuj1", "i3HmbHac0eQp6D3iC2QMRtNIs7a2", "Et7CFW8jkcX1GRjETBPcGsy42kW2",
    "zFJFxFXuJhTjC7oRQ30u8UJIFLF2"
]

groups = {
    'Control Group': control_group_ids,
    'Intervention Group': intervention_group_ids
}

true_articles = ['A2', 'A4', 'A6']
fake_articles = ['A1', 'A3', 'A5']

with pd.ExcelWriter('output.xlsx') as writer:
    for group_name, group_ids in groups.items():
        all_rows = []
        for user_id, user_values in user_data.items():
            if user_id in group_ids:
                rows = []
                first_entry_for_user = True  # A flag to keep track of first entry for a user
                for date, articles in user_values.items():
                    for url_encoded, metrics in articles['articles'].items():
                        url = url_encoded.replace('%', '/').replace(' ', '.')
                        row = [
                            url,
                            metrics['article_id'],
                            metrics['article_read'],
                            metrics['avg_reading_time'],
                            metrics['click_rate'],
                            metrics['current_article_bias'],
                            metrics['current_reading_time'],
                            convert_time_to_seconds(metrics['current_reading_time']),
                            metrics['dominant_bias'],
                            metrics['page_title'],
                            metrics['scroll_rate'],
                            metrics['source'],
                            user_id if first_entry_for_user else ''  # Only add UserID for the first row of this user
                        ]
                        first_entry_for_user = False  # Set the flag to false after the first entry
                        rows.append(row)
                all_rows.extend(rows + [[''] * 13])  # Adding an empty row between different UserIDs

        df_group = pd.DataFrame(all_rows, columns=[
            "URL", "article_id", "article_read", "avg_reading_time", "click_rate",
            "current_article_bias", "current_reading_time", "current_reading_time (seconds)",
            "dominant_bias", "page_title", "scroll_rate", "source", "UserID"
        ])

        # Add a space before the "UserID" column
        df_group['source'] = df_group['source'].astype(str) + ' '

        df_group['current_reading_time (seconds)'] = pd.to_numeric(df_group['current_reading_time (seconds)'], errors='coerce')

        # Calculate the desired metrics
        avg_reading_time = df_group['current_reading_time (seconds)'].mean()
        avg_true_article_time = df_group[df_group['article_id'].isin(true_articles)]['current_reading_time (seconds)'].mean()
        avg_fake_article_time = df_group[df_group['article_id'].isin(fake_articles)]['current_reading_time (seconds)'].mean()

        # Print the metrics to the Excel sheet
        df_metrics = pd.DataFrame({
            'Metric': ['Average Current Reading Time', 'Average Current Reading Time for True Articles', 'Average Current Reading Time for Fake Articles'],
            'Value': [avg_reading_time, avg_true_article_time, avg_fake_article_time]
        })

        # Write to Excel file
        df_group.to_excel(writer, sheet_name=f'{group_name} Data', index=False)
        df_metrics.to_excel(writer, sheet_name=f'{group_name} Metrics', index=False)

print("Data written to output.xlsx")


Data written to output.xlsx


In [ ]:
import json
import pandas as pd

def convert_time_to_seconds(time_str):
    mins, secs = 0, 0
    if "Mins" in time_str:
        mins = int(time_str.split("Mins")[0].strip())
        secs = int(time_str.split("Mins")[1].split("Secs")[0].strip().split(":")[1])
    else:
        secs = int(time_str.split("Secs")[0].strip())
    return mins * 60 + secs

with open('database1.json', 'r') as file:
    data = json.load(file)

user_data = data['user_session_data_test']
user_log_data = data.get('user_log', {})

control_group_ids = [
    "izZlxECbEvbZbaQsiTC2IppTR1D2", "w1bQfQVpS0e2bdMS0mgXpSHqZew2", "Uah5QqbULfOwCEwuKqEmRv7ELx42",
    "l1JBi3rjs7eEzKECNW02rhKkzI12", "kt6wMrsVN1YNwymFsZCPha4eG6w1", "gSznvtvnoeMV3bvm9aMP0uwlUhl2",
    "ZbmqqGFDwmZXkjR9lxJRB8Y6tmd2", "U2HjmAv7KCcfQVHsr7ofVPdaTm82", "n3mtRWzHDRSm9NYqoGxyyM7zyuQ2",
    "Vv3J8PwWWeSWErREMVtrxwkP8rF3", "ogFBk6o71cWw7TQejwgzYWYdetK2", "YOlWWzDFhyUJuCCazPl7vMdPLj23",
    "wrGFkJdPKwNNcOIDSPmN62VGIzC2", "g6RotBsUG0fQVfkSampcMTaWdTE3", "2blzSKV1tkcSFq2f8opLrcM5iP33",
    "EfVmI4iVenXsMAX392YBkIgysH23", "lJNN7pxBDhTulyLFLr70FghZLfN2", "6X2UCYgJSBV0dFWD0yWYodEmNKu2"
]

intervention_group_ids = [
    "3YPGBN3vD3TIqg3Jlrxgu16gj4K3", "ehQq99p6woP538JE31l8s0lcBr62", "NMUVcKxsfbT76avSebXt6fmsKw02",
    "iVMz6gl0vkWNZuTJ64bUXEj1AJm1", "K9IJNZxN3OUNiOn6kFuia77Io9m2", "ttrSxxnr5ROCWCynB2kC2cJgkEr1",
    "ZXkLCiyTgyXJGmL5dXF7Pip5Tpk2", "qf7BdXXKWTNisXiY3dNGynktezC3", "6AfBZNSFoDYtFZoK3v6IoDzYGX23",
    "DMRsCjYfw3Ui2WY7baPfiliLmYm1", "yOwa5JFnOFcPhY68B9JfQ8FXhqN2", "RXrptwB3hBeMx77a0g09or8T7zk2",
    "4N71ZXMaaTcJbEmeUAg1KrZlQNA3", "iXcMJZtl9wXfvb5elAKio2Nrn0D2", "OIf29ayJNJUCcBRExfWG27W9iCh1",
    "okJYh5CBOrbVuzpP7dLOiBWxGuj1", "i3HmbHac0eQp6D3iC2QMRtNIs7a2", "Et7CFW8jkcX1GRjETBPcGsy42kW2",
    "zFJFxFXuJhTjC7oRQ30u8UJIFLF2"
]

groups = {
    'Control Group': control_group_ids,
    'Intervention Group': intervention_group_ids
}


with pd.ExcelWriter('output.xlsx') as writer:
    for group_name, group_ids in groups.items():
        all_rows = []
        for user_id, user_values in user_data.items():
            if user_id in group_ids:
                rows = []
                known_urls = set()
                first_entry_for_user = True

                additional_urls = set()  # Store additional URLs for this user

                for date, articles in user_values.items():
                    for url_encoded, metrics in articles['articles'].items():
                        url = url_encoded.replace('%', '/').replace(' ', '.')
                        known_urls.add(url)
                        row = [
                            url,
                            metrics['article_id'],
                            metrics['article_read'],
                            metrics['avg_reading_time'],
                            metrics['click_rate'],
                            metrics['current_article_bias'],
                            metrics['current_reading_time'],
                            convert_time_to_seconds(metrics['current_reading_time']),
                            metrics['dominant_bias'],
                            metrics['page_title'],
                            metrics['scroll_rate'],
                            metrics['source'],
                            user_id if first_entry_for_user else ''  # Only add UserID for the first row of this user
                        ]
                        first_entry_for_user = False
                        rows.append(row)

                # Check for additional URLs from user_log
                user_log = user_log_data.get(user_id, {})

                for date, log in user_log.items():
                    for url_encoded in log.get('articles', []):
                        url = url_encoded.replace('%', '/').replace(' ', '.')
                        if url not in known_urls:
                            additional_urls.add(url)

                # Add the additional URLs as a column to the first row (if they exist)
                if rows and additional_urls:
                    rows[0].append(' | '.join(additional_urls))
                elif additional_urls:
                    rows.append([''] * 12 + [user_id] + [' | '.join(additional_urls)])
                else:
                    # If no additional URLs, just append an empty space
                    if rows:
                        rows[0].append('')

                all_rows.extend(rows + [[''] * 14])

        df_group = pd.DataFrame(all_rows, columns=[
            "URL", "article_id", "article_read", "avg_reading_time", "click_rate",
            "current_article_bias", "current_reading_time", "current_reading_time (seconds)",
            "dominant_bias", "page_title", "scroll_rate", "source", "UserID", "Additional URLs"
        ])

        df_group['source'] = df_group['source'].astype(str) + ' '
        df_group['current_reading_time (seconds)'] = pd.to_numeric(df_group['current_reading_time (seconds)'], errors='coerce')

        # Calculate the desired metrics
        avg_reading_time = df_group['current_reading_time (seconds)'].mean()
        avg_true_article_time = df_group[df_group['article_id'].isin(true_articles)]['current_reading_time (seconds)'].mean()
        avg_fake_article_time = df_group[df_group['article_id'].isin(fake_articles)]['current_reading_time (seconds)'].mean()

        df_metrics = pd.DataFrame({
            'Metric': ['Average Current Reading Time', 'Average Current Reading Time for True Articles', 'Average Current Reading Time for Fake Articles'],
            'Value': [avg_reading_time, avg_true_article_time, avg_fake_article_time]
        })

        df_group.to_excel(writer, sheet_name=f'{group_name} Data', index=False)
        df_metrics.to_excel(writer, sheet_name=f'{group_name} Metrics', index=False)

print("Data written to output.xlsx")


Data written to output.xlsx


In [ ]:
import json
import pandas as pd

def convert_time_to_seconds(time_str):
    mins, secs = 0, 0
    if "Mins" in time_str:
        mins = int(time_str.split("Mins")[0].strip())
        secs = int(time_str.split("Mins")[1].split("Secs")[0].strip().split(":")[1])
    else:
        secs = int(time_str.split("Secs")[0].strip())
    return mins * 60 + secs

with open('database.json', 'r') as file:
    data = json.load(file)

user_data = data['user_session_data_test']
user_log_data = data.get('user_log', {})

control_group_ids = [
    "m7y3ik7wygMz18y53q24cRQ0IUm1", "MNLmUNyuoAhsctnRCDyDFUqdk5O2", "izZlxECbEvbZbaQsiTC2IppTR1D2",
    "w1bQfQVpS0e2bdMS0mgXpSHqZew2", "Uah5QqbULfOwCEwuKqEmRv7ELx42", "l1JBi3rjs7eEzKECNW02rhKkzI12",
    "kt6wMrsVN1YNwymFsZCPha4eG6w1", "gSznvtvnoeMV3bvm9aMP0uwlUhl2", "ZbmqqGFDwmZXkjR9lxJRB8Y6tmd2",
    "U2HjmAv7KCcfQVHsr7ofVPdaTm82", "n3mtRWzHDRSm9NYqoGxyyM7zyuQ2", "Vv3J8PwWWeSWErREMVtrxwkP8rF3",
    "ogFBk6o71cWw7TQejwgzYWYdetK2", "YOlWWzDFhyUJuCCazPl7vMdPLj23", "wrGFkJdPKwNNcOIDSPmN62VGIzC2",
    "g6RotBsUG0fQVfkSampcMTaWdTE3", "2blzSKV1tkcSFq2f8opLrcM5iP33", "EfVmI4iVenXsMAX392YBkIgysH23",
    "lJNN7pxBDhTulyLFLr70FghZLfN2", "5iUhzT0PbYXwnwqxtV5noQBi9F63", "yRsUxHOBhrfNgCBI5qGRStwjmro1",
    "jVo6tCQtDBX87xspQtRi6CYKFlA3", "niurytWFevdaI7ChIhrnEikzcqG3", "kEkfvtxjKdadzzgY2pUbDdBCTBt1",
    "5uu5xIx85QOQQHNwg0gzw3UUiBC2", "HfC9NyJvCEQHuuE9xRkVPZrlmLx2", "ImsJdRqkckRP7TbNrWMB4JMRSJZ2",
    "kYcjrZMDvXW3ZOdxVNN24WcaUQ43", "p3MbZA6YoIUEjhsPskBJunxy9Ro1", "xiPdEybZlCY9A5BXwjfTpdkSIZn1",
    "WFU91jLayNV8GMzfEpqqy08P7d13", "yhoSSD2xIeWH8gnQ616SwtqEFKm2", "TCxKEnBv8zdsCF92blKac9rz8JB3",
]

intervention_group_ids = [
    "O5zQix01toRNApLU1xzl8m2IUoa2", "3YPGBN3vD3TIqg3Jlrxgu16gj4K3", "ehQq99p6woP538JE31l8s0lcBr62",
    "NMUVcKxsfbT76avSebXt6fmsKw02", "iVMz6gl0vkWNZuTJ64bUXEj1AJm1", "K9IJNZxN3OUNiOn6kFuia77Io9m2",
    "ttrSxxnr5ROCWCynB2kC2cJgkEr1", "ZXkLCiyTgyXJGmL5dXF7Pip5Tpk2", "qf7BdXXKWTNisXiY3dNGynktezC3",
    "6AfBZNSFoDYtFZoK3v6IoDzYGX23", "DMRsCjYfw3Ui2WY7baPfiliLmYm1", "yOwa5JFnOFcPhY68B9JfQ8FXhqN2",
    "RXrptwB3hBeMx77a0g09or8T7zk2", "4N71ZXMaaTcJbEmeUAg1KrZlQNA3", "iXcMJZtl9wXfvb5elAKio2Nrn0D2",
    "OIf29ayJNJUCcBRExfWG27W9iCh1", "okJYh5CBOrbVuzpP7dLOiBWxGuj1", "i3HmbHac0eQp6D3iC2QMRtNIs7a2",
    "Et7CFW8jkcX1GRjETBPcGsy42kW2", "zFJFxFXuJhTjC7oRQ30u8UJIFLF2", "CUFJTwC3SfhFuqCVc3m9r0nRsUp1",
    "A7v2UgpD3QSG8RSzChNfX5GLha13", "vIdx8TsPc4YvjehcrYFnMWskRs73", "YZZ3iX1klhOgkdm8iWFqTFgXTe03",
    "DBoGxzxrMNg35jvii1JkjFFOzQ42", "u8GeP9nHKWeyRHkkJG6yjUGMnYY2", "V1V9NB0peEOpBgsp8c8eTOYmc6Z2",
    "a6bTHKiHC8c1Iwtk8fBVGUJVN993", "oNu8tY2ELVhmHhUyAwz4jq1GRiQ2", "jxjNj9lGrrOSvLg3gxOY5CnxyG03",
    "7Jnatc1vczRidXKPNsM5LlsVan73", "a7VP7VwxbkhH177rO6iBDEgE68v2", "bJxW3getIbhJBC1qmBUttjRnlrf1",
    "zQPKgOM0HsPZLcHcmIxOu6EG8yI3", "vD5cfxpwyCZ7mWSkst4eaQ7r9xj2", "Xf29cjqMJjO7sQViRDF8x0D20VX2",
    "afDRYXLOKIVnnWQM5XrTKJzklz42", "PNAHYU6Q7ROTcHMQnXou3Tkhj1b2", "CyRaDUzppeXweovlWZpUpIpaHth2",
    "IxtPpUJaN3XSwGxKhk6nwVdRzYC3", "CNw5mvk8V2cazrHJdms6PhmRBTu1",
]

groups = {
    'Control Group': control_group_ids,
    'Intervention Group': intervention_group_ids
}


with pd.ExcelWriter('output.xlsx') as writer:
    for group_name, group_ids in groups.items():
        all_rows = []
        for user_id, user_values in user_data.items():
            if user_id in group_ids:
                rows = []
                known_urls = set()
                first_entry_for_user = True  # A flag to keep track of first entry for a user

                for date, articles in user_values.items():
                    for url_encoded, metrics in articles['articles'].items():
                        url = url_encoded.replace('%', '/').replace(' ', '.')
                        known_urls.add(url)
                        row = [
                            url,
                            metrics['article_id'],
                            metrics['article_read'],
                            metrics['avg_reading_time'],
                            metrics['click_rate'],
                            metrics['current_article_bias'],
                            metrics['current_reading_time'],
                            convert_time_to_seconds(metrics['current_reading_time']),
                            metrics['dominant_bias'],
                            metrics['page_title'],
                            metrics['scroll_rate'],
                            metrics['source'],
                            user_id if first_entry_for_user else ''  # Only add UserID for the first row of this user
                        ]
                        first_entry_for_user = False  # Set the flag to false after the first entry
                        rows.append(row)

                # Check for additional URLs from user_log
                additional_urls = set()
                user_log = user_log_data.get(user_id, {})

                for date, log in user_log.items():
                    for url_encoded in log.get('articles', []):
                        url = url_encoded.replace('%', '/').replace(' ', '.')
                        if url not in known_urls:
                            additional_urls.add(url)

                # Append the additional URLs to the rows (if any exist)
                if additional_urls:
                    # Creating the 'Additional URLs' row
                    additional_url_row = ['Additional URLs:'] + list(additional_urls) + [''] * (11 - len(additional_urls))
                    if len(rows) == 0:  # If no other rows exist for this user, append the user_id
                        additional_url_row.append(user_id)
                    else:
                        additional_url_row.append('')  # Add an empty space for the UserID column
                    rows.append(additional_url_row)

                all_rows.extend(rows + [[''] * 13])  # Adding an empty row between different UserIDs

        df_group = pd.DataFrame(all_rows, columns=[
            "URL", "article_id", "article_read", "avg_reading_time", "click_rate",
            "current_article_bias", "current_reading_time", "current_reading_time (seconds)",
            "dominant_bias", "page_title", "scroll_rate", "source", "UserID"
        ])

        df_group.to_excel(writer, sheet_name=group_name, index=False)

print("Data written to output.xlsx")


Data written to output.xlsx
